In [1]:
import json
import pandas as pd
import numpy as np
import requests

ACCOUNT_ID = "001-001-2265866-001"
API_TOKEN = "591691d91a571232b46b9b87e74145ba-4fc5bc3427b4eb7d79c3501316fdffa0"

API_URL = "api-fxtrade.oanda.com"
STREAM_URL = "stream-fxtrade.oanda.com"

PRICING_PATH = f"/v3/accounts/{ACCOUNT_ID}/pricing"

query = {"instruments": "EUR_USD"} 
headers = {"Authorization": "Bearer "+ API_TOKEN}

response = requests.get("https://"+API_URL+PRICING_PATH, headers=headers, params=query) 

response.json()

{'time': '2024-08-10T18:13:48.300457568Z',
 'prices': [{'type': 'PRICE',
   'time': '2024-08-09T20:59:05.192155128Z',
   'bids': [{'price': '1.09146', 'liquidity': 500000},
    {'price': '1.09145', 'liquidity': 2500000},
    {'price': '1.09144', 'liquidity': 2000000},
    {'price': '1.09143', 'liquidity': 5000000},
    {'price': '1.09140', 'liquidity': 10000000},
    {'price': '1.09137', 'liquidity': 10000000}],
   'asks': [{'price': '1.09165', 'liquidity': 500000},
    {'price': '1.09166', 'liquidity': 500000},
    {'price': '1.09167', 'liquidity': 2000000},
    {'price': '1.09168', 'liquidity': 2000000},
    {'price': '1.09169', 'liquidity': 5000000},
    {'price': '1.09171', 'liquidity': 10000000},
    {'price': '1.09174', 'liquidity': 10000000}],
   'closeoutBid': '1.09137',
   'closeoutAsk': '1.09174',
   'status': 'non-tradeable',
   'tradeable': False,
   'quoteHomeConversionFactors': {'positiveUnits': '1.00000000',
    'negativeUnits': '1.00000000'},
   'instrument': 'EUR_USD'}

In [2]:
from OandaAPI import OandaAPI
oa = OandaAPI(account_id=ACCOUNT_ID, api_token=API_TOKEN)

gbp_jpy_df = oa.show_granularity()
shit_df = oa.historical_data(currency_pair='EUR_USD', start_date='08/09/2023', end_date='08/04/2024', granularity='D', as_pandas=True)
gbp_jpy_df

,Description
Granularity Value,
S5,"5 second candlesticks, minute alignment"
S10,"10 second candlesticks, minute alignment"
S15,"15 second candlesticks, minute alignment"
S30,"30 second candlesticks, minute alignment"
M1,"1 minute candlesticks, minute alignment"
M2,"2 minute candlesticks, hour alignment"
M4,"4 minute candlesticks, hour alignment"
M5,"5 minute candlesticks, hour alignment"
M10,"10 minute candlesticks, hour alignment"


In [4]:
eur_usd_df = oa.load_candlestick_datafame(currency_pair='EUR_USD', start_date='01/01/2014', end_date='01/03/2024', granularity='M1')
eur_usd_df

{'instrument': 'EUR_USD', 'granularity': 'M1', 'candles': []}
"['complete'] not found in axis"


KeyError: 'DateTime'

In [11]:
# Resampling data to 4-hour intervals
eur_usd_df = eur_usd_df.resample("4h").agg({
        "Open": "first",
        "High": "max",
        "Low": "min",
        "Close": "last",
        "Volume": "sum"})

eur_usd_df 


,Open,High,Low,Close,Volume
DateTime,,,,,
2014-01-01 20:00:00+00:00,1.37525,1.37738,1.37426,1.37642,5106
2014-01-02 00:00:00+00:00,1.37644,1.37756,1.37506,1.37662,7980
2014-01-02 04:00:00+00:00,1.37662,1.37662,1.37258,1.37322,12078
2014-01-02 08:00:00+00:00,1.37322,1.37424,1.36844,1.36895,28248
2014-01-02 12:00:00+00:00,1.36896,1.36896,1.36296,1.36580,35755
...,...,...,...,...,...
2023-12-29 04:00:00+00:00,1.10654,1.10764,1.10611,1.10660,8179
2023-12-29 08:00:00+00:00,1.10653,1.10776,1.10444,1.10764,15038
2023-12-29 12:00:00+00:00,1.10766,1.10807,1.10427,1.10474,23766
